In [0]:
USE weather;

In [0]:
----------------------------
-- CURRENT WEATHER SNAPSHOT
----------------------------
CREATE OR REPLACE TABLE weather.gold_current_weather_snapshot
USING DELTA
LOCATION '/mnt/data/gold/weather_current_snapshot'
AS
  SELECT
    l.city_id,
    l.city_name,
    l.country,
    w.localtime,
    w.temperature_celsius,
    w.feels_like_celsius,
    w.weather_description,
    w.humidity,
    w.wind_speed_kph,
    w.pressure_mb,
    w.precipitation_mm
  FROM weather.silver_weather_cleaned w
  JOIN weather.silver_dim_location l
  ON w.city_id = l.city_id
  WHERE w.ingestion_date = (SELECT MAX(ingestion_date) FROM weather.silver_weather_cleaned)
  ORDER BY l.city_id;

In [0]:
-----------------------------
-- GENERAL AGGREGATES BY CITY
-----------------------------
CREATE OR REPLACE TABLE weather.gold_aggregates_by_city
USING DELTA
LOCATION '/mnt/data/gold/weather_aggregates_by_city'
AS
  SELECT
    l.city_id,
    l.city_name,
    -- Temperature
    AVG(w.temperature_celsius) avg_temperature,
    MIN(w.temperature_celsius) min_temperature,
    MAX(w.temperature_celsius) max_temperature,
    -- Humidity
    AVG(w.humidity) avg_humidity,
    MIN(w.humidity) min_humidity,
    MAX(w.humidity) max_humidity,
    -- Wind Speed
    AVG(w.wind_speed_kph) avg_wind_speed,
    MIN(w.wind_speed_kph) min_wind_speed,
    MAX(w.wind_speed_kph) max_wind_speed

  FROM weather.silver_weather_cleaned w
  JOIN weather.silver_dim_location l
  ON w.city_id = l.city_id
  GROUP BY l.city_id, l.city_name
  ORDER BY l.city_id;

In [0]:
--------------------------------
-- 7 DAYS MOVING AVERAGES REPORT
--------------------------------
CREATE OR REPLACE TABLE weather.gold_7days_moving_avgs
USING DELTA
LOCATION '/mnt/data/gold/weather_7days_moving_avgs'
AS
  WITH weather_data AS (
    SELECT
      l.city_id,
      l.city_name,
      CAST(DATE_TRUNC('day', w.observation_time_utf) AS DATE) observation_date,
      w.temperature_celsius,
      w.humidity,
      w.wind_speed_kph,
      w.precipitation_mm
    FROM weather.silver_weather_cleaned w
    JOIN weather.silver_dim_location l
    ON w.city_id = l.city_id
  )
  SELECT
    city_id,
    city_name,
    observation_date,
    ROUND(AVG(temperature_celsius) OVER (PARTITION BY city_id ORDER BY observation_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) avg_temperature_7_days,
    ROUND(AVG(humidity) OVER (PARTITION BY city_id ORDER BY observation_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) avg_humidity_7_days,
    ROUND(AVG(wind_speed_kph) OVER (PARTITION BY city_id ORDER BY observation_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) avg_wind_speed_7_days,
    ROUND(SUM(precipitation_mm) OVER (PARTITION BY city_id ORDER BY observation_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) total_precipitation_7_days
  FROM weather_data;

In [0]:
---------------------------------------
-- WEATHER EVENTS SUMMARY (LAST 30 DAYS)
---------------------------------------
CREATE OR REPLACE TABLE weather.gold_weather_events_summary
USING DELTA
LOCATION '/mnt/data/gold/weather_events_summary'
AS
  SELECT
    l.city_name,
    'Last 30 Days' AS analys_period,
    SUM(CASE WHEN w.temperature_celsius > 30 THEN 1 ELSE 0 END) hot_days_count,
    SUM(CASE WHEN w.temperature_celsius < 15 THEN 1 ELSE 0 END) cold_days_count,
    SUM(CASE WHEN w.temperature_celsius < 0 THEN 1 ELSE 0 END) frost_days_count,
    SUM(CASE WHEN w.precipitation_mm > 1 THEN 1 ELSE 0 END) rainy_days_count
  FROM weather.silver_weather_cleaned w
  JOIN weather.silver_dim_location l
  ON w.city_id = l.city_id
  WHERE w.observation_time_utf >= current_date() - INTERVAL 30 DAYS
  GROUP BY l.city_name;